In [4]:
import pandas as pd
import numpy as np

#insert file_path for the data
#file_path = 'H:/Documents/Merged data ANN.xlsx'
file_path = 'C:/Users/Vince Van Ham/OneDrive/Documenten/kuleuven/Thesis/Merged data ANN.xlsx'
#read data, and quickly check if it is correctly read in
data= pd.read_excel(file_path)
# Load the data from Excel

#data.drop(columns=["date"], inplace=True)


In [10]:
import requests

def count_amenity(lat, lon, amenity, radius=500):
    base_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    node["amenity"="{amenity}"](around:{radius},{lat},{lon});
    out;
    """
    response = requests.get(base_url, params={'data': query})
    data = response.json()
    elements = data.get('elements', [])
    return len(elements)

# Example usage for a specific location in Paris
lat, lon = 48.8566, 2.3522  # Latitude and longitude for a central point in Paris
print(f"Bars near {lat}, {lon}: {count_amenity(lat, lon, 'bar')}")
print(f"Restaurants near {lat}, {lon}: {count_amenity(lat, lon, 'restaurant')}")


Bars near 48.8566, 2.3522: 51
Restaurants near 48.8566, 2.3522: 174


# used on a small dataset

In [10]:
import pandas as pd
import requests

# Assuming 'short_data' is your DataFrame containing Airbnb listings with 'latitude' and 'longitude' columns.
# Here's a placeholder for your actual DataFrame loading:
# short_data = pd.read_excel('/path/to/your/airbnb/data.xlsx')

def count_amenity(lat, lon, amenity, radius=500):
    base_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    node["amenity"="{amenity}"](around:{radius},{lat},{lon});
    out;
    """
    response = requests.get(base_url, params={'data': query})
    data = response.json()
    elements = data.get('elements', [])
    return len(elements)


def count_transit(lat, lon, transit_type, radius=500):
    base_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    node["{transit_type}"~"subway_entrance|bus_stop"](around:{radius},{lat},{lon});
    out;
    """
    response = requests.get(base_url, params={'data': query})
    data = response.json()
    elements = data.get('elements', [])
    return len(elements)


# Function to apply to each row in the DataFrame
def add_amenity_counts(row):
    lat, lon = row['latitude'], row['longitude']
    row['amount_nearby_bars'] = count_amenity(lat, lon, 'bar')
    row['amount_nearby_restaurants'] = count_amenity(lat, lon, 'restaurant')
    row['nearby_subway_entrances'] = count_transit(lat, lon, 'railway', 'subway_entrance')
    row['nearby_bus_stops'] = count_transit(lat, lon, 'highway', 'bus_stop')
    return row

    return row

# Apply the function to each row in the DataFrame
# Note: This operation might take some time for a large dataset due to API request processing.
short_data = short_data.apply(add_amenity_counts, axis=1)

# Display the first few rows of the updated DataFrame to verify the new columns
print(short_data.head(20))

# Optionally, save the updated DataFrame back to an Excel file
# short_data.to_excel('/path/to/save/updated_airbnb_data.xlsx', index=False)



   host_response_time  host_response_rate  host_is_superhost  \
0                   4                   1              False   
1                   4                   1              False   
2                   4                   1              False   
3                   4                   1              False   
4                   4                   1               True   
5                   4                   1               True   
6                   4                   1               True   
7                   4                   1               True   
8                   1                   4              False   
9                   1                   4              False   

   host_listings_count  host_total_listings_count  host_identity_verified  \
0                    1                          1                    True   
1                    1                          1                    True   
2                    1                          1               

In [12]:
import pandas as pd
import requests

# Assuming 'short_data' is your DataFrame
# Example for loading your DataFrame:
# short_data = pd.read_excel('/path/to/your/airbnb/data.xlsx')

def count_amenity(lat, lon, query_type, value, radius=500):
    base_url = "http://overpass-api.de/api/interpreter"
    if query_type == "amenity":
        query_filter = f'["{query_type}"="{value}"]'
    else:  # For transit types like subway entrances and bus stops, use a different tag
        query_filter = f'["{query_type}"~"{value}"]'
        
    query = f"""
    [out:json];
    node{query_filter}(around:{radius},{lat},{lon});
    out;
    """
    response = requests.get(base_url, params={'data': query})
    data = response.json()
    elements = data.get('elements', [])
    return len(elements)

# Function to apply to each row in the DataFrame
def add_amenity_counts(row):
    lat, lon = row['latitude'], row['longitude']
    row['amount_nearby_bars'] = count_amenity(lat, lon, 'amenity', 'bar')
    row['amount_nearby_restaurants'] = count_amenity(lat, lon, 'amenity', 'restaurant')
    row['nearby_subway_entrances'] = count_amenity(lat, lon, 'railway', 'subway_entrance')
    row['nearby_bus_stops'] = count_amenity(lat, lon, 'highway', 'bus_stop')
    return row

# Apply the function to each row in the DataFrame
# Note: This operation might take some time for a large dataset due to API request processing.
short_data = short_data.apply(add_amenity_counts, axis=1)

# Display the first few rows of the updated DataFrame to verify the new columns
print(short_data.head(20))

# Optionally, save the updated DataFrame back to an Excel file
# short_data.to_excel('/path/to/save/updated_airbnb_data.xlsx', index=False)



   host_response_time  host_response_rate  host_is_superhost  \
0                   4                   1              False   
1                   4                   1              False   
2                   4                   1              False   
3                   4                   1              False   
4                   4                   1               True   
5                   4                   1               True   
6                   4                   1               True   
7                   4                   1               True   
8                   1                   4              False   
9                   1                   4              False   

   host_listings_count  host_total_listings_count  host_identity_verified  \
0                    1                          1                    True   
1                    1                          1                    True   
2                    1                          1               

In [16]:
!pip install geopandas
!pip install rtree


     ---------------------------------------- 1.1/1.1 MB 14.1 MB/s eta 0:00:00
     --------------------------------------- 22.9/22.9 MB 46.7 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 89.3 MB/s eta 0:00:00
     ---------------------------------------- 6.1/6.1 MB 55.3 MB/s eta 0:00:00


In [16]:
import geopandas as gpd
from shapely.geometry import Point

# Assuming 'short_data' is your original DataFrame with latitude and longitude
# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(short_data, geometry=gpd.points_from_xy(short_data.longitude, short_data.latitude))

# Create a spatial index
sindex = gdf.sindex

def count_nearby_listings(row, gdf, radius_meters=500):
    # Convert the radius in meters to degrees (approximation)
    radius_degrees = radius_meters / 111320  # Rough conversion factor for degrees to meters
    
    # Find possible matches with rtree index, which returns the index of possible matches
    possible_matches_index = list(sindex.intersection(row.geometry.buffer(radius_degrees).bounds))
    
    # Filter actual matches from possible matches
    possible_matches = gdf.iloc[possible_matches_index]
    actual_matches = possible_matches[possible_matches.geometry.distance(row.geometry) <= radius_degrees]
    
    # Exclude the row itself from the count
    nearby_count = actual_matches.shape[0] - 1
    return nearby_count

# Apply the function to count nearby listings for each listing
gdf['nearby_airbnbs_count'] = gdf.apply(count_nearby_listings, axis=1, gdf=gdf)

# Convert back to a regular DataFrame if necessary, or continue using as a GeoDataFrame
updated_data = pd.DataFrame(gdf.drop(columns='geometry'))

print(updated_data.head(10))


   host_response_time  host_response_rate  host_is_superhost  \
0                   4                   1              False   
1                   4                   1              False   
2                   4                   1              False   
3                   4                   1              False   
4                   4                   1               True   
5                   4                   1               True   
6                   4                   1               True   
7                   4                   1               True   
8                   1                   4              False   
9                   1                   4              False   

   host_listings_count  host_total_listings_count  host_identity_verified  \
0                    1                          1                    True   
1                    1                          1                    True   
2                    1                          1               

## used on the full dataset

In [18]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Assuming 'data' is your original DataFrame with latitude and longitude
# First, ensure 'data' is loaded correctly, e.g., data = pd.read_excel('path/to/your/data.xlsx')

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.longitude, data.latitude))

# Create a spatial index for the GeoDataFrame
sindex = gdf.sindex

def count_nearby_listings(row, gdf, radius_meters=500):
    # Convert the radius in meters to degrees (approximation)
    # Note: This approximation works well for small distances (e.g., within a few kilometers)
    radius_degrees = radius_meters / 111320  # Rough conversion factor for degrees to meters
    
    # Find possible matches with rtree index, which returns indices of possible matches
    possible_matches_index = list(sindex.intersection(row.geometry.buffer(radius_degrees).bounds))
    
    # Filter actual matches from possible matches
    possible_matches = gdf.iloc[possible_matches_index]
    actual_matches = possible_matches[possible_matches.geometry.distance(row.geometry) <= radius_degrees]
    
    # Exclude the row itself from the count
    nearby_count = actual_matches.shape[0] - 1
    return nearby_count

# Apply the function to count nearby listings for each listing
gdf['nearby_airbnbs_count'] = gdf.apply(count_nearby_listings, axis=1, gdf=gdf)

# If you want to work with a regular DataFrame (excluding the geometry column)
updated_data = pd.DataFrame(gdf.drop(columns='geometry'))

# Display the first few rows of the updated DataFrame to verify the new column
print(updated_data.head(20))


    host_response_time  host_response_rate  host_is_superhost  \
0                    4                   1              False   
1                    4                   1              False   
2                    4                   1              False   
3                    4                   1              False   
4                    4                   1               True   
5                    4                   1               True   
6                    4                   1               True   
7                    4                   1               True   
8                    1                   4              False   
9                    1                   4              False   
10                   1                   4              False   
11                   1                   4              False   
12                   1                   4              False   
13                   1                   4              False   
14                   1   

## filtered for unique locations full dataset

In [17]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Assuming 'data' is your DataFrame

# Step 1: Identify unique locations
unique_locations = data[['latitude', 'longitude']].drop_duplicates()
unique_gdf = gpd.GeoDataFrame(unique_locations, geometry=gpd.points_from_xy(unique_locations.longitude, unique_locations.latitude))

# Create a spatial index for the unique GeoDataFrame
sindex_unique = unique_gdf.sindex

def count_nearby_listings(row, gdf, sindex, radius_meters=500):
    radius_degrees = radius_meters / 111320
    possible_matches_index = list(sindex.intersection(row.geometry.buffer(radius_degrees).bounds))
    possible_matches = gdf.iloc[possible_matches_index]
    actual_matches = possible_matches[possible_matches.geometry.distance(row.geometry) <= radius_degrees]
    nearby_count = actual_matches.shape[0] - 1
    return nearby_count

# Step 2: Calculate Nearby Airbnb Count for Unique Locations
unique_gdf['nearby_airbnbs_count'] = unique_gdf.apply(count_nearby_listings, axis=1, gdf=unique_gdf, sindex=sindex_unique)

# Step 3: Merge the Counts Back to the Original Dataset
# First, convert the GeoDataFrame back to a DataFrame for easier merging
unique_counts = pd.DataFrame(unique_gdf.drop(columns='geometry'))

# Merge the unique counts back into the original 'data' DataFrame based on latitude and longitude
data = pd.merge(data, unique_counts, on=['latitude', 'longitude'], how='left')

print(data.head(20))


    host_response_time  host_response_rate  host_is_superhost  \
0                    4                   1              False   
1                    4                   1              False   
2                    4                   1              False   
3                    4                   1              False   
4                    4                   1               True   
5                    4                   1               True   
6                    4                   1               True   
7                    4                   1               True   
8                    1                   4              False   
9                    1                   4              False   
10                   1                   4              False   
11                   1                   4              False   
12                   1                   4              False   
13                   1                   4              False   
14                   1   

In [14]:
import pandas as pd
import requests

# Assuming 'short_data' is your DataFrame
# Example for loading your DataFrame:
# short_data = pd.read_excel('/path/to/your/airbnb/data.xlsx')

def count_amenity(lat, lon, query_type, value, radius=500):
    base_url = "http://overpass-api.de/api/interpreter"
    if query_type == "amenity":
        query_filter = f'["{query_type}"="{value}"]'
    else:  # For transit types like subway entrances and bus stops, use a different tag
        query_filter = f'["{query_type}"~"{value}"]'
        
    query = f"""
    [out:json];
    node{query_filter}(around:{radius},{lat},{lon});
    out;
    """
    response = requests.get(base_url, params={'data': query})
    data = response.json()
    elements = data.get('elements', [])
    return len(elements)

# Function to apply to each row in the DataFrame
def add_amenity_counts(row):
    lat, lon = row['latitude'], row['longitude']
    row['amount_nearby_bars'] = count_amenity(lat, lon, 'amenity', 'bar')
    row['amount_nearby_restaurants'] = count_amenity(lat, lon, 'amenity', 'restaurant')
    row['nearby_subway_entrances'] = count_amenity(lat, lon, 'railway', 'subway_entrance')
    row['nearby_bus_stops'] = count_amenity(lat, lon, 'highway', 'bus_stop')
    return row

# Apply the function to each row in the DataFrame
# Note: This operation might take some time for a large dataset due to API request processing.
data = data.apply(add_amenity_counts, axis=1)

# Display the first few rows of the updated DataFrame to verify the new columns
print(data.head(20))

# Optionally, save the updated DataFrame back to an Excel file
data.to_excel('C:/Users/Vince Van Ham/OneDrive/Documenten/kuleuven/Thesis/data osm features.xlsx', index=False)


KeyboardInterrupt: 

In [18]:
import pandas as pd
import requests

# Assuming 'short_data' is your DataFrame
# Example loading your DataFrame:
# short_data = pd.read_excel('/path/to/your/airbnb/data.xlsx')

def count_amenity(lat, lon, query_type, value, radius=500):
    base_url = "http://overpass-api.de/api/interpreter"
    if query_type == "amenity":
        query_filter = f'["{query_type}"="{value}"]'
    else:
        query_filter = f'["{query_type}"~"{value}"]'
        
    query = f"""
    [out:json];
    node{query_filter}(around:{radius},{lat},{lon});
    out;
    """
    response = requests.get(base_url, params={'data': query})
    data = response.json()
    elements = data.get('elements', [])
    return len(elements)

# Group by unique latitude and longitude, then apply the count_amenity function
unique_coords = data.groupby(['latitude', 'longitude']).first().reset_index()

# Apply counts for each unique pair of coordinates
unique_coords['amount_nearby_bars'] = unique_coords.apply(lambda row: count_amenity(row['latitude'], row['longitude'], 'amenity', 'bar'), axis=1)
unique_coords['amount_nearby_restaurants'] = unique_coords.apply(lambda row: count_amenity(row['latitude'], row['longitude'], 'amenity', 'restaurant'), axis=1)
unique_coords['nearby_subway_entrances'] = unique_coords.apply(lambda row: count_amenity(row['latitude'], row['longitude'], 'railway', 'subway_entrance'), axis=1)
unique_coords['nearby_bus_stops'] = unique_coords.apply(lambda row: count_amenity(row['latitude'], row['longitude'], 'highway', 'bus_stop'), axis=1)

# Merge the count data back into the original dataframe based on latitude and longitude
merged_data = pd.merge(data, unique_coords[['latitude', 'longitude', 'amount_nearby_bars', 'amount_nearby_restaurants', 'nearby_subway_entrances', 'nearby_bus_stops']], on=['latitude', 'longitude'], how='left')

print(merged_data.head(20))



KeyboardInterrupt: 

In [ ]:
# Optionally, save the updated DataFrame back to an Excel file
merged_data.to_excel('C:/Users/Vince Van Ham/OneDrive/Documenten/kuleuven/Thesis/data osm features.xlsx', index=False)